In [2]:


import astropy
print('astropy',astropy.__version__ )
from spectral_cube import SpectralCube      # This is a handy package for working with 3D data cubes
from astropy.coordinates import SkyCoord
from reproject import reproject_interp      
from reproject.mosaicking import find_optimal_celestial_wcs 
import regions
import reproject
print('reproject',reproject.__version__)
import spectral_cube
print('spectral_cube',spectral_cube.__version__)
import numpy as np                          
import pylab                                
import matplotlib 
import matplotlib.gridspec as gridspec                                                                                             
import scipy
import astropy.io.fits as fits                                                          
from astropy.wcs import WCS                 
from astropy import units as u              
import pyspeckit as psk   
import astrodendro
import matplotlib.colors as colors
from matplotlib import pyplot as plt
# Suppress warnings we don't care about:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
%matplotlib inline
from astropy.convolution import Gaussian1DKernel
#%matplotlib widget
import radio_beam
import copy

astropy 5.1.dev153+gb740594dc
reproject 0.8
spectral_cube 0.6.1.dev22+g003ef16


In [3]:
files=['NGC253.HCO+_4-3_mJy.pbcor.fits','NGC253.HCN_4-3_mJy.pbcor.fits',"NGC253.H13CN_4-3_mJy.pbcor.fits","HCN_J1-0.cube.fits",'HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits']     
files=['NGC253.HCO+_4-3_mJy.pbcor.fits','NGC253.HCN_4-3_mJy.pbcor.fits',"NGC253.H13CN_4-3_mJy.pbcor.fits","HCN_J1-0.cube.fits",'HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits']     
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)   
    #header = fits.getheader(file)
    if i ==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    
    pathInf = 'NGC253.HCO+_4-3_mJy.pbcor.fits'
    HI = SpectralCube.read(pathInf).header
    #Nres = (HI["BMAJ"]+HI["BMIN"])/2*3.5*10**6/180*np.pi*u.pc#pc beam res, project everything to same pc res
    Nres=3*u.pc
    dist=3.5*10**6
    res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc
    print(res,HI["BMAJ"],HI["BMIN"])
    beam = radio_beam.Beam(major=res, minor=res, pa=HI["BPA"]*u.deg)
    print(beam)
    sc.allow_huge_operations=True
    sc = sc.convolve_to(beam)   
    gal="NGC253"
    print(np.shape(sc))
    reheader = copy.deepcopy(sc.header)
    if sc.header['cdelt1']>0:
        pix_x    = (res/5.).to(u.degree).value
        origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
        print(pix_x,origin_x,"A")
    else:
        pix_x    = -1.*(res/5.).to(u.degree).value
        origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
        print(pix_x,origin_x,"B")
    if sc.header['cdelt2']>0:
        pix_y    = (res/5.).to(u.degree).value
        origin_y = sc.latitude_extrema[0].to(u.degree).value
        print(pix_y,origin_y,"c")
    else:
        pix_y    = -1.*(res/5.).to(u.degree).value
        origin_y = sc.latitude_extrema[1].to(u.degree).value
        print(pix_y,origin_y,"d")
    if gal=='NGC253':
        npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
        npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
        print(npix_x,npix_y)
    elif gal=='GC':
        npix_x   = int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
        npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
        print(npix_x,npix_y)

    reheader['cdelt1'] = pix_x
    reheader['cdelt2'] = pix_y
    reheader['naxis1'] = npix_x
    reheader['naxis2'] = npix_y
    reheader['crval1'] = origin_x
    reheader['crval2'] = origin_y
    reheader['crpix1'] = 0
    reheader['crpix2'] = 0
    try:
        del reheader['lonpole']
        del reheader['latpole']
        del reheader['wcsaxes']
    except:
        pass

    # regrid cube to target pixel size
    print(reheader)
    scN = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
        
        
    y, x = scN.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')
    scW = scN.wcs[:][:][0]
    
    
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = scN[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1. *u.km / u.s, 501. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    #print(sc_K_kms.wcs)
    #print(sc_K_kms.world[:,0,0][0])
    #print(header)
    if i ==0:
        vel = np.arange(0,500,2.5)*u.km/u.s
        
        
        fwhm_factor = np.sqrt(8*np.log(2))

        NGC_HCO_reprojectGauss = sc_K_kms.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
        del sc_K_kms

        NGC_HCO_Plus_Reproject_Interp = NGC_HCO_reprojectGauss.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 250 range 
        del NGC_HCO_reprojectGauss
        NGC_HCO_Plus_Reproject_Interp.write('NGC253.HCO+_4-3_mJy_Reproject.pbcor.fits',overwrite=True)
        
    if i==1:
        fwhm_factor = np.sqrt(8*np.log(2))

        NGC_HCN_reprojectGauss = sc_K_kms.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
        del sc_K_kms

        NGC_HCN_Plus_Reproject_Interp = NGC_HCN_reprojectGauss.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 250 range 
        del NGC_HCN_reprojectGauss
        
        NGC_HCN_Plus_Reproject_Interp.write('NGC253_HCN.cube.reproject.mJy-kms.pbcor.fits',overwrite=True)
        del NGC_HCN_Plus_Reproject_Interp
        
    if i==2:
        
        NGC_H13CN = sc_K_kms.unmasked_copy()#This one needs reprojection

del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_H13CN.allow_huge_operations=True

HCOPH = NGC_HCO_Plus_Reproject_Interp.header


print(NGC_HCO_Plus)
print(NGC_H13CN)

print(type(NGC_HCO_Plus))
print(type(NGC_H13CN))

print(type(NGC_HCO_Plus[0]))
print(type(NGC_H13CN[0]))

print(type(NGC_HCO_Plus[0][0]))
print(type(NGC_H13CN[0][0]))

del NGC_HCO_Plus #rem from mem


NGC_H13CN_reproject = NGC_H13CN.reproject(HCOPH,use_memmap=True)  
del NGC_H13CN #rem from mem
del HCOPH

fwhm_factor = np.sqrt(8*np.log(2))

NGC_HCN_reprojectGauss = NGC_H13CN_reproject.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
del NGC_H13CN_reproject
      

NGC_HC13N_reproject_interp = NGC_HCN_reprojectGauss.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del NGC_HCN_reprojectGauss
NGC_HC13N_reproject_interp.write('NGC_H13CN.cube.reproject.mJy-kms.pbcor.fits',overwrite=True)      
del NGC_HC13N_reproject_interp         
print("done")        

4.911066815407056e-05 deg 4.722222222222e-05 3.611111111111e-05
Beam: BMAJ=0.17679840535465402 arcsec BMIN=0.17679840535465402 arcsec BPA=-73.32 deg
-1.2796754075622423 -1.2796754075622423
1.0315638844900257e-09 2.582009513047944e-10


KeyboardInterrupt: 

In [ ]:
#The first three cubes haves been reporjected

In [ ]:
files=['NGC253.HCO+_4-3_mJy.pbcor.fits','NGC253.HCN_4-3_mJy.pbcor.fits',"NGC253.H13CN_4-3_mJy.pbcor.fits","HCN_J1-0.cube.fits",'HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits']     
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i ==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1. *u.km / u.s, 501. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    
    
    if i==0:
        vel = np.arange(0,500,2.5)*u.km/u.s
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==3:
        
        NGC_HCN_J1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_HCN_J1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'  #Change from F2F because it seems to be the same thing. See https://www.aanda.org/articles/aa/pdf/2006/05/aa3818-05.pdf which says that it means "Sampled in freq, expressed in freq". Side note, some comments in the header may be incorrect, such as incorrect units (comments say m/s where it should be km/s)


del NGC_HCO_Plus #rem from mem

NGC_HCN_J1_reproject = NGC_HCN_J1.reproject(HCOPH,use_memmap=True)  
del NGC_HCN_J1 #rem from mem
del HCOPH
NGC_HCN_J1_reproject_interp = NGC_HCN_J1_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del NGC_HCN_J1_reproject
NGC_HCN_J1_reproject_interp.write('NGC_HCN_J1.cube.reproject.Jy-kms.pbcor.fits',overwrite=True)      
del NGC_HCN_J1_reproject_interp
print("done")        

In [ ]:
files=['NGC253.HCO+_4-3_mJy.pbcor.fits','NGC253.HCN_4-3_mJy.pbcor.fits',"NGC253.H13CN_4-3_mJy.pbcor.fits","HCN_J1-0.cube.fits",'HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits']     
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i ==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1. *u.km / u.s, 501. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    
    
    if i ==0:
        vel = np.arange(0,500,2.5)*u.km/u.s
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==4:
        
        NGC_HCOP_J1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_HCOP_J1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

NGC_HCOP_J1_reproject = NGC_HCOP_J1.reproject(HCOPH,use_memmap=True)  
del NGC_HCOP_J1 #rem from mem
del HCOPH
NGC_HCOP_J1_reproject_interp = NGC_HCOP_J1_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del  NGC_HCOP_J1_reproject
NGC_HCOP_J1_reproject_interp.write('NGC_HCOP_J1_0.cube.reproject.Jy-kms.pbcor.fits',overwrite=True)      
del NGC_HCOP_J1_reproject_interp
print("done")              

In [ ]:
files=['NGC253.HCO+_4-3_mJy.pbcor.fits','NGC253.HCN_4-3_mJy.pbcor.fits',"NGC253.H13CN_4-3_mJy.pbcor.fits","HCN_J1-0.cube.fits",'HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits']     
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i ==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1. *u.km / u.s, 501. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    
    
    
    
    if i ==0:
        vel = np.arange(0,500,2.5)*u.km/u.s
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
        
    if i==5:
        NGC_H13CN_J1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_H13CN_J1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

NGC_H13CN_J1_reproject = NGC_H13CN_J1.reproject(HCOPH,use_memmap=True)  
del NGC_H13CN_J1 #rem from mem
del HCOPH
NGC_H13CN_J1_reproject_interp = NGC_H13CN_J1_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del NGC_H13CN_J1_reproject
NGC_H13CN_J1_reproject_interp.write('NGC_H13CN_J1_0.cube.reproject.Jy-kms.pbcor.fits',overwrite=True)  
del NGC_H13CN_J1_reproject_interp        
    
     
print("done")             

In [ ]:
files=['NGC253.HCO+_4-3_mJy.pbcor.fits','NGC253.HCN_4-3_mJy.pbcor.fits',"NGC253.H13CN_4-3_mJy.pbcor.fits","HCN_J1-0.cube.fits",'HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits']     
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i ==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1. *u.km / u.s, 501. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    
    
    if i ==0:
        vel = np.arange(0,500,2.5)*u.km/u.s
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==6:
        NGC_CO_J3_2 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_CO_J3_2.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

NGC_CO_J3_2_reproject = NGC_CO_J3_2.reproject(HCOPH,use_memmap=True)  
del NGC_CO_J3_2 #rem from mem
del HCOPH
fwhm_factor = np.sqrt(8*np.log(2))

CO32_reprojectGauss = NGC_CO_J3_2_reproject.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
del NGC_CO_J3_2_reproject

CO32_reproject_interp = CO32_reprojectGauss.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 250 range 
print(CO32_reproject_interp.world[:][0][0],CO32_reprojectGauss.world[:][0][0])
del CO32_reprojectGauss
CO32_reproject_interp = CO32_reproject_interp.hdu
NGC_CO_J3_2_reproject_interp = SpectralCube.read(CO32_reproject_interp)



NGC_CO_J3_2_reproject_interp.write('NGC_CO_J3_2.cube.reproject.Jy-kms.pbcor.fits',overwrite=True)      
del NGC_CO_J3_2_reproject_interp  

print("done")         

In [ ]:
files=['NGC253.HCO+_4-3_mJy.pbcor.fits','NGC253.HCN_4-3_mJy.pbcor.fits',"NGC253.H13CN_4-3_mJy.pbcor.fits","HCN_J1-0.cube.fits",'HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits']     
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i ==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1. *u.km / u.s, 501. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    
    
    if i ==0:
        vel = np.arange(0,500,2.5)*u.km/u.s
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==7:
        CCH_N1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
CCH_N1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

CCH_N1_reproject =CCH_N1.reproject(HCOPH,use_memmap=True)  
del CCH_N1 #rem from mem
del HCOPH
CCH_N1_reproject_interp = CCH_N1_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del CCH_N1_reproject
CCH_N1_reproject_interp.write('CCH_N1.cube.reproject.Jy-kms.pbcor.fits',overwrite=True)   
del CCH_N1_reproject_interp         
print("done")            

In [ ]:
files = pylab.array(['NGC253.HCO+_4-3_mJy.pbcor.fits','NGC253.HCN_4-3.pbcor.fits','NGC253.H13CN_4-3.pbcor.fits','HCN_J1-0.cube.fits','HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits',"HC3N_11_10.fits"])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i ==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1. *u.km / u.s, 501. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    
    
    if i ==0:
        vel = np.arange(0,500,2.5)*u.km/u.s
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==8:
        HC3N_J11 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
HC3N_J11.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

HC3N_J11_reproject =HC3N_J11.reproject(HCOPH,use_memmap=True)  
del HC3N_J11 #rem from mem
del HCOPH
HC3N_J11_reproject_interp = HC3N_J11_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del HC3N_J11_reproject
HC3N_J11_reproject_interp.write('HC3N_J11.cube.reproject.Jy-kms.pbcor.fits',overwrite=True) 
del HC3N_J11_reproject_interp          
print("done")        

In [ ]:

      
files = pylab.array(['NGC253.HCO+_4-3_mJy.pbcor.fits', "H13CO_Plus_1_0.fits"])


for i,file in enumerate(files):
                    
    sc = SpectralCube.read(file)  
    
    cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1. *u.km / u.s, 501. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    
    
    if i ==0:
        vel = np.arange(0,500,2.5)*u.km/u.s
        
        NGC_HCO_Plus = sc_K_kms
        
    if i==1:
        H13CoPJ1_0 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
H13CoPJ1_0.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
print(HCOPH)
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

H13CoPJ1_0_reproject = H13CoPJ1_0.reproject(HCOPH,use_memmap=True)  
del H13CoPJ1_0 #rem from mem
del HCOPH
H13CoPJ1_0_reproject_interp = H13CoPJ1_0_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del H13CoPJ1_0_reproject
H13CoPJ1_0_reproject_interp.write('H13CoPJ1_0.cube.reproject.Jy-kms.pbcor.fits',overwrite=True)      
del H13CoPJ1_0_reproject_interp        
           
print("done")        

In [ ]:
files = pylab.array(['NGC253.HCO+_4-3_mJy.pbcor.fits', "SiO_2_1.fits"])


for i,file in enumerate(files):
                    
    sc = SpectralCube.read(file)  
    
    cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1. *u.km / u.s, 501. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    
    
    if i ==0:
        vel = np.arange(0,500,2.5)*u.km/u.s
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==1:
        SiJ2_1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
SiJ2_1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

SiJ2_1_0_reproject = SiJ2_1.reproject(HCOPH,use_memmap=True)  
del SiJ2_1 #rem from mem
del HCOPH
SiJ2_1_reproject_interp = SiJ2_1_0_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del SiJ2_1_0_reproject
SiJ2_1_reproject_interp.write('SiJ2_1.cube.reproject.Jy-kms.pbcor.fits',overwrite=True)       

del SiJ2_1_reproject_interp        
          
    

    
    
    
print("done")

In [ ]:
files = pylab.array(['NGC253.HCO+_4-3_mJy.pbcor.fits','ngc253_850_micron_dust.fits'])



for i,file in enumerate(files):
    
    if i ==0:
        sc = SpectralCube.read(file) 
        cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
        cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')
        scW = sc.wcs[:][:][0]
        raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
        raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
        subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    if i==1:
        subcube = spectral_cube.Projection.from_hdu(fits.open(file)[0])
    #print(subcube.hdu.header)
    
    
    #Put in the right system
    
    subcube.allow_huge_operations=True
    
    
    
    if i ==0:
        vel, dec, ra = subcube.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = subcube.moment0()
    if i==1:
        Cont = subcube
        

        
del dec
del ra
Cont.allow_huge_operations=True
#HCOPH = NGC_HCO_Plus.header
HCOPH = NGC_HCO_Plus.hdu.header
#HCOPH['NAXIS'] = 2
print(HCOPH)
#print([HCOPH['NAXIS{0}'.format(i + 1)] for i in range(HCOPH['NAXIS'])][::-1])
print([Cont.hdu.header['NAXIS{0}'.format(i + 1)] for i in range(Cont.hdu.header['NAXIS'])][::-1])
print(np.shape(NGC_HCO_Plus))
del NGC_HCO_Plus #rem from mem

Cont_reproject =Cont.reproject(HCOPH)  
del HCOPH

Cont_reproject.write('Continuum_Reproject.fits',overwrite=True) 
print(np.shape(Cont_reproject))
del Cont_reproject          
print("done")        

In [ ]:



files = pylab.array(['HCOp_J1-0.cube.fits','Continuum_Reproject.fits'])



for i,file in enumerate(files):
    
    if i ==0:
        sc = SpectralCube.read(file) 
        cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
        cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')
        scW = sc.wcs[:][:][0]
        raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
        raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
        subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    if i==1:
        subcube = spectral_cube.Projection.from_hdu(fits.open(file)[0])
    #print(subcube.hdu.header)
    
    
    #Put in the right system
    
    subcube.allow_huge_operations=True
    
    
    
    if i ==0:
        vel, dec, ra = subcube.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = subcube.moment0()
    if i==1:
        Cont = subcube
        

Cont.allow_huge_operations=True
#HCOPH = NGC_HCO_Plus.header
HCOPH = NGC_HCO_Plus.hdu.header
#HCOPH['NAXIS'] = 2
print(HCOPH)
#print([HCOPH['NAXIS{0}'.format(i + 1)] for i in range(HCOPH['NAXIS'])][::-1])
print([Cont.hdu.header['NAXIS{0}'.format(i + 1)] for i in range(Cont.hdu.header['NAXIS'])][::-1])
print(np.shape(NGC_HCO_Plus))
del NGC_HCO_Plus #rem from mem

Cont_reproject =Cont.reproject(HCOPH)  
del HCOPH

Cont_reproject.write('Continuum_Reproject_V2.fits',overwrite=True) 
print(np.shape(Cont_reproject))
del Cont_reproject          
print("done")        

In [ ]:
#Fix frequency info

files = pylab.array(['HCOp_J1-0.cube.fits','HCN_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits',"SiO_2_1.fits","H13CO_Plus_1_0.fits",'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits','NGC253.H13CN_4-3.pbcor.fits','NGC253.HCO+_4-3.pbcor.fits','NGC253.HCN_4-3.pbcor.fits'])

filesR = pylab.array(['NGC_HCOP_J1_0.cube.reproject.Jy-kms.pbcor.fits','NGC_HCN_J1.cube.reproject.Jy-kms.pbcor.fits','NGC_H13CN_J1_0.cube.reproject.Jy-kms.pbcor.fits','NGC_CO_J3_2.cube.reproject.Jy-kms.pbcor.fits','SiJ2_1.cube.reproject.Jy-kms.pbcor.fits','H13CoPJ1_0.cube.reproject.Jy-kms.pbcor.fits','CCH_N1.cube.reproject.Jy-kms.pbcor.fits','HC3N_J11.cube.reproject.Jy-kms.pbcor.fits','NGC_H13CN.cube.reproject.mJy-kms.pbcor.fits','NGC_HCO_Plus.cube.mJy-kms.pbcor.fits','NGC253_HCN.cube.reproject.mJy-kms.pbcor.fits'])
for i in range(len(filesR)):
    try:
        freq = fits.getheader(files[i])["RESTFREQ"]
    except:
        freq = fits.getheader(files[i])["RESTFRQ"]
    c=SpectralCube.read(filesR[i])
    ch=c.hdu.header
    try:
        
        ch["RESTFREQ"] =freq
        ch["RESTFRQ"] =freq
        print(ch["RESTFREQ"],"L",ch["RESTFRQ"])
        #print(ch,"DONNNNN1")
    except:
        ch["RESTFRQ"] =freq
        print(ch["RESTFRQ"],"P",ch["RESTFREQ"])
        #print(ch,"DONNNNN")
    
    
    #c.write(filesR[i],overwrite=True)
    
    #CN =  SpectralCube(data=c.hdu.data, wcs=c.wcs,header=ch)*u.K
    #print(ch.cards)
    chp = fits.Header.fromstring(ch.tostring())
    
    CN =  SpectralCube(data=c.hdu.data, wcs=c.wcs,header=chp)*c[0][0][0].unit
    FN = "Fixed_"+filesR[i]
    print(CN.header["RESTFRQ"],CN.header.cards["RESTFREQ"])
    #CN.header.cards["RESTFREQ"] = CN.header["RESTFRQ"]
    print(CN.header["RESTFRQ"],CN.header["RESTFREQ"])
    #print(CN.wcs)
    CN.write(FN,overwrite=True)
    

CMZpathsR = pylab.array(['cmz_HCOp.cube.mJy-kms.pbcor.fits','cmz_HCN.cube.reproject.mJy-kms.pbcor.fits','cmz_H13CN.cube.reproject.mJy-kms.pbcor.fits',"Spliced_CO_V2.fits",'cmz_H13COp.cube.reproject.mJy-kms.pbcor.fits'])


for i in range(len(CMZpathsR)):
    try:
        freq = fits.getheader(files[i])["RESTFREQ"]
    except:
        freq = fits.getheader(files[i])["RESTFRQ"]
    c=SpectralCube.read(CMZpathsR[i])
    ch=c.hdu.header
    try:
        
        ch["RESTFREQ"] =freq
        ch["RESTFRQ"] =freq
        print(ch["RESTFREQ"],"L",ch["RESTFRQ"])
    except:
        ch["RESTFRQ"] =freq
        print(ch["RESTFRQ"],"P",ch["RESTFREQ"])
        
    
    
    #c.write(filesR[i],overwrite=True)
    
    CN =  SpectralCube(data=c.hdu.data, wcs=c.wcs,header=ch)*c[0][0][0].unit
    FN = "Fixed_"+CMZpathsR[i]
    print("C",CN.header["RESTFRQ"],CN.header["RESTFREQ"])
    CN.write(FN,overwrite=True)